In [1]:
from sys import byteorder
from array import array
from struct import pack

import os
import sys
import pyaudio
import wave

from pydub import AudioSegment
from pydub.silence import split_on_silence
import IPython.display as ipd


from six.moves import xrange
import numpy as np
import time
import math
#import alsaaudio

In [2]:
import numpy as np
import os
from scipy import signal
import matplotlib.pyplot as plt
import scipy

In [4]:
import scipy.signal as sg
from scipy.io import wavfile
import time

In [5]:
def is_silent(sound_data, threshold = 1300):
    "Returns 'True' if below the 'silent' threshold"
    return max(sound_data) < threshold

def normalize(sound_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in sound_data)

    r = array('h')
    for i in sound_data:
        r.append(int(i*times))
    return r

def trim(sound_data, threshold = 1300):
    "Trim the blank spots at the start and end"
    def _trim(sound_data, threshold = 1300):
        sound_data_started = False
        r = array('h')

        for i in sound_data:
            if not sound_data_started and abs(i) > threshold:
                sound_data_started = True
                r.append(i)

            elif sound_data_started:
                r.append(i)
        return r

    # Trim to the left
    sound_data = _trim(sound_data, threshold = threshold)

    # Trim to the right
    sound_data.reverse()
    sound_data = _trim(sound_data, threshold = threshold)
    sound_data.reverse()
    return sound_data

def add_silence(sound_data, pad, rate = 16000):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    pad_list = [0 for i in xrange(int(pad))]
    r = array('h', pad_list)
    r.extend(sound_data)
    r.extend(pad_list)
    return r

def record(threshold = 1300, chunk_size = 1024, 
           rate = 16000, channels =1, 
           silent_limit_control = 12,
           format_ = pyaudio.paInt16,
           n_records = 2):
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    data = []
    sample_width = []
    print('n_silence: ', silent_limit_control)
    for n_r in range(n_records):
        print('current n_silence: ', silent_limit_control[n_r])
        stream = p.open(format = format_, 
                        channels = channels, 
                        rate = rate,
                        input = True, 
                        output = True,
                        frames_per_buffer = chunk_size)

        num_silent = 0
        sound_data_started = False

        r = array('h')

        while 1:
            # little endian, signed short
            sound_data = array('h', stream.read(chunk_size))
            print(max(sound_data))
            if byteorder == 'big':
                sound_data.byteswap()
            r.extend(sound_data)

            silent = is_silent(sound_data, threshold)

            if silent and sound_data_started:
                num_silent += 1
            elif not silent and not sound_data_started:
                sound_data_started = True

            if sound_data_started and num_silent > silent_limit_control[n_r]:
                break

        sample_width.append(p.get_sample_size(format_))
        data.append(r)
        stream.stop_stream()
        stream.close()
        s = ipd.Audio(r, rate = rate, autoplay = True)
        ipd.display(s)
        
        
    p.terminate()
    
    for i in range(n_records):
        r = data[i]
        
        print('len: ', len(r))
        np_r = np.array(r, dtype = type(r))
        print('sample_width: ', sample_width[i])
        print('normalizing data')
        r = normalize(r)
        print('trimming data')
        r = trim(r, threshold)
        print('adding silence')
        print('before adding silence data type: ', type(r))
        r = add_silence(r, 0.25, rate = rate)
        print('after adding silence data type: ', type(r))
        time.sleep(3)
    #print('stream rms: ', np.sqrt(np.mean(r**2)))
    #return sample_width, r
    return None, None


In [6]:
def get_top_two_data(data, stride = 4000, window = 12000, sample_rate = 16000):
    
    data = np.array(data, dtype = type(data))
    print('data: ', data.shape[0], window, stride)
    if data.shape[0] < window:
        raise ValueError('Data Should be atleast 12000 frames or 0.75 sec in length')
    chunk_list = []
    rms_list = []
    k = 1
    #time.sleep(2)
    for i in range(0, data.shape[0], stride):
        
        chunk  = data[i: i+window]
        rms = np.sqrt(np.mean(chunk ** 2))
        rms_list.append(rms)
        ch_max = chunk.max()
        ch_min = chunk.min()
        print('chunk : ', str(k), ' rms: ', rms, 'min: ', ch_min, 'max: ', ch_max)
        print('difference: ', ch_max- ch_min)
        k += 1
        sound = ipd.Audio(chunk, rate = sample_rate, autoplay = True)
        ipd.display(sound)
        time.sleep(0.5)
    #print(rms)
    #idxs = np.argsort(rms)[-2:]
    #print(np.argsort(rms))
    
    #return chunk_list[]
        
    

In [7]:
def record_to_file(path = './demo_.wav', threshold = 1300, 
                   chunk_size = 1024, rate = 16000, 
                   channels =1, silent_limit_control = 12,
                   format_ = pyaudio.paInt16,
                   stride = 4000, window = 12000):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record(threshold, chunk_size, rate, 
                                channels, silent_limit_control, format_)
    """
    print('IN record_to_file')
    print('Type of recorded data: ', type(data))
    
    sound = ipd.Audio(data, rate = rate, autoplay = True)
    ipd.display(sound)
    
    print('Recording Happen Successfully')
    print('we can check for rms before saving')
    time.sleep(2.0)
    get_top_two_data(data, stride = stride, window = window, sample_rate = rate)
    
    
    #data_ = pack('<' + ('h'*len(data)), *data)
    print('Saving recorded data to: ', path)
    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(rate)
    wf.writeframes(np.array(data))
    wf.close()
    print('Recorded data saved and now returning data: ', np.array(data).shape)
    return data
    """

    
    
def check_audio_duration(fname = './word_1.wav', rate = 16000):
    f = wave.open(fname, 'r')
    frames = f.getnframes()
    rate = f.getframerate()
    duration = (frames / float(rate)) * 1000
    #print('sample rate = ', rate)
    print('seconds = ', duration/1000)
    print('IN check_audio_duration')
    sound = ipd.Audio(fname, rate = rate, autoplay = True)
    ipd.display(sound)
    

In [8]:
def clip_for_silence(wave_file = './demo.wav', out_file1 = './word1.wav', 
                     out_file2 = './word2.wav',
                     pad = 2000, file_format = 'wav', r = None, id_ = 'sr'):        
    average_loudness_l = []
    loudness_l = []
    max_amp_l = []
    ratio_l = []
    n_segments_l = []
    i = 0
    
    print('Reading File using wave.open: ', wave_file)
    f = wave.open(wave_file, 'r')
    frames = f.getnframes()
    print('n_frames: ', frames)
    rate = f.getframerate()
    print('sample rate: ', rate)
    duration = (frames / float(rate)) * 1000
    print('duration: ', duration)
    #print('sample rate = ', rate)
    #print('seconds = ', duration/1000)
    
    print('pad : ', pad)
    
    print(type(r))
    #r = add_silence(r, pad)
    
    
    #"""
    if id_ == 'dcr':
        min_silence_len = min(600, int(duration/4.0))    
    #for data_collector_recorder
    
    elif id_ == 'sr':
        min_silence_len = min(300, int(duration/3.0))
    #
    #print('min_silence lenth should be : ',min_silence_len)

    #ipd.Audio(filename = wav_file, rate = 16000)
    print('Reading file using AudioSegment.from_wave: ', wave_file)
    sound = AudioSegment.from_wav(wave_file)
    average_loudness = sound.rms
    average_loudness_l.append(average_loudness)
    #print('average_loudness : ', average_loudness)
    print('sound rms: ', average_loudness)
    print('stream rms: ', np.sqrt(np.mean(np.array(r, dtype = type(r))**2)))


    loudness = sound.dBFS
    loudness_l.append(loudness)
    #print('loudness : ', loudness)

    max_amp = sound.max_possible_amplitude
    max_amp_l.append(max_amp)
    #print('log max_amp  : ', np.log10(max_amp)*20)

    ratio = 20 * np.log10(average_loudness / max_amp)
    ratio_l.append(ratio)
    #print('ratio - avg loudness / max_amp  : ', ratio)

    n_segments = -1
    s_t = np.round(33.0, 2)
    return r
    """
    audio_segments = split_on_silence(sound,
                                      min_silence_len = min_silence_len,
                                      keep_silence = keep_silence,
                                      silence_thresh = -1 * s_t)
    n_segments = len(audio_segments)
    max_idx = 0
    rms_list = []
    
    
    print("type of f ", type(f))
    for k, seg in enumerate(audio_segments):
        print('j = ',k,' rms: ', seg.rms, 'dBFS: ', seg.dBFS, 'max_amp: ', seg.max_possible_amplitude)
        print('\t ratio: ', 20*np.log10(seg.rms/seg.max_possible_amplitude))
        print("type of segment", type(seg))
        print("number of frames: ",(np.array(seg).shape[0])*16)
        rms_list.append(seg.rms)
        #print()
    rms_list = np.array(rms_list)
    #idx = np.argmax(rms_list)
    #idxs = np.arange(rms_list.shape[0])
    idxs = np.sort(np.argsort(rms_list)[-2:])
    #print('number of segments : ', n_segments, 'max idx = ', idx)
    
    #if n_segments == 1:


    audio_segments[idxs[0]].export(out_file1, format = file_format)
    audio_segments[idxs[1]].export(out_file2, format = file_format)
    """

In [9]:
def start_recorder(fname_path = './recordings/', 
                    keep_silence = 20, threshold = 1300, 
                    chunk_size = 1024, 
                    rate = 16000, channels = 1, 
                    silent_limit_control = 6,
                    window = 12000,
                    stride = 4000):
    id_ = 'sr'
    format_ = pyaudio.paInt16
    wave_list = []
    if not os.path.exists(fname_path):
                os.makedirs(fname_path)
    
    if not os.path.exists(fname_path + '/word/'):
                os.makedirs(fname_path + '/word/')
    for i in range(1):
        print("please speak a words into the microphone")
        time.sleep(0.5)
        record_fname = fname_path + '/demo_' + str(i+1) + '.wav'
        #r = record_to_file(record_fname, threshold, chunk_size, rate, channels, silent_limit_control, format_)
        record_to_file(record_fname, 
                       threshold,
                       chunk_size,
                       rate, 
                       channels,
                       silent_limit_control,
                       format_,
                       window = window,
                       stride = stride)
        """
        print(type(r))
        
        sound = AudioSegment.from_wav(record_fname)
        print("shape:  ", np.array(sound).shape)
        print(len(np.array(sound).shape))
        check_audio_duration(record_fname, rate = rate)
        time.sleep(1)
        r_ = clip_for_silence(wave_file = record_fname, r = r)
        print('IN start_recorder')
        sound = ipd.Audio(r_, rate = rate, autoplay = True)
        ipd.display(sound)
        """

In [15]:
start_recorder(threshold = 1000,
               chunk_size = 2048,
               window = 8000,
               stride = 4000,
               silent_limit_control = [2, 6]  )

please speak a words into the microphone
n_silence:  [2, 6]
current n_silence:  2


OSError: [Errno -9997] Invalid sample rate

In [ ]:
16384/16
